In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

X = pd.concat([train.drop(['click'],axis=1),test])
print(X.shape)

X = X.sort_values('time')
X = X.reset_index(drop=True)

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')

X = X.sort_values('time')
X = X.reset_index(drop=True)

print(X.shape)

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 34)
(1041674, 45)
(1041674, 35)


# Step 1, change data type

In [3]:
for col in X.columns:
    if X[col].dtype == 'float64':
        X[col] = X[col].astype(utils.set_type(X[col],'float'))
    if X[col].dtype == 'int64':
        X[col] = X[col].astype(utils.set_type(X[col],'int'))
        
print(sys.getsizeof(X)/1024**3)

0.785621888935566


# Step 2, sort user tags, to make sure it is ordered consistently

In [4]:
def tags_sort(x):
    try:
        return ','.join(sorted(list(set(x.split(',')) -set(['']))))
    except:
        return x
X['user_tags'] = X['user_tags'].apply(tags_sort)
nan_index = X['user_tags'][X['user_tags'].isnull()].index
X['user_tags'] = X['user_tags'].fillna('notFound')
X['user_tags'] = le.fit_transform(X['user_tags'])
X.loc[nan_index,'user_tags'] = np.nan




# Step 3, creative_size, encoded by 'creative_width' and 'creative_height'

In [5]:
X['creative_size'] = X['creative_width'].astype(str) + '*' + X['creative_height'].astype(str)
X.drop(['creative_width', 'creative_height'],axis=1,inplace=True)

# Step 4, drop those only has 1 unique value

In [6]:
drop_col = []
for col in X.columns:
    if X[col].nunique() == 1:
        drop_col.append(col)
        
print(drop_col)

X.drop(drop_col,axis=1,inplace=True)

['creative_is_js', 'creative_is_voicead', 'app_paid']


# Step 5, combine 'city_province'

In [7]:
X['city_province'] = X['city'].astype(str)+'_' + X['province'].astype(str)
X['city_province'] = le.fit_transform(X['city_province'])
X.drop(['city', 'province'],axis=1,inplace=True)

In [8]:
le = LabelEncoder()
not_categorical = ['instance_id','time', 'datetime','click']
categorical_cols = list(set(X.columns) - set(not_categorical))
print(len(categorical_cols))


for col in tqdm(categorical_cols):
    if X[col].dtype == 'object':
        X[col] = X[col].fillna('FillNAValue')
        X[col] = le.fit_transform(X[col])
    else:
        pass
#         X[col] = X[col].fillna(-100000)
#         X[col] = le.fit_transform(X[col])

  0%|          | 0/28 [00:00<?, ?it/s]

28


100%|██████████| 28/28 [00:07<00:00,  3.63it/s]


In [9]:
user_list = [
['user_tags'],
['user_tags','city_province'],
['user_tags','model','os_name','make','devtype'],
['city_province','user_tags','model','nnt','devtype','os_name','osv','make','os','carrier'],
['user_tags','model','nnt','devtype','os_name','osv','make','os','carrier']
]

ads_list = [
['adid'],
['adid','creative_size'],
['adid','orderid','campaign_id','creative_id','creative_type','creative_tp_dnf','creative_size'],
['advert_id','advert_industry_inner','advert_name','campaign_id','creative_id','creative_type','creative_tp_dnf','creative_size'],['adid','advert_id','orderid','advert_industry_inner','advert_name','campaign_id','creative_id','creative_type','creative_tp_dnf','creative_size']
]

channel_list = [
['app_id'],
['app_id','inner_slot_id'],
['f_channel','inner_slot_id'],
['app_cate_id','f_channel','app_id','inner_slot_id']
]

time_list = [
[],
['time_hour']
]



user = ['city_province','user_tags','model','nnt','devtype','os_name','osv','make','os','carrier']
ads = ['adid','advert_id','orderid','advert_industry_inner','advert_name','campaign_id','creative_id','creative_type','creative_tp_dnf','creative_has_deeplink','creative_is_jump','creative_is_download','creative_size']

channel = ['app_cate_id','f_channel','app_id','inner_slot_id',]

# Define Function List

In [10]:
count_func_list = [count,cumulative_count,reverse_cumulative_count]
time_next_func_list = [time_to_n_next,time_to_n_previous]
count_time_func_list = [count_in_previous_n_time_unit,count_in_next_n_time_unit]

# FillNa

In [11]:
# le = LabelEncoder()
# #X['user_tags'] =  X['user_tags'].fillna('naNotFound')
# # X['user_tags'] = le.fit_transform(X['user_tags'])

# for col in tqdm(categorical_cols):
#     if X[col].dtype == 'object':
#         X[col] = X[col].fillna('FillNAValue')
#         pass
#     else:
#         X[col] = X[col].fillna(-100000)
#     X[col] = le.fit_transform(X[col])

# Aggregation

In [12]:
def get_new_name(cob_col,func):
    return  'user{}_ads{}_channel{}_hour{}_'.format(*cob_col) +func.__name__

In [13]:
def auto_generate_func(X,user_list,ads_list,channel_list,time_list,
                    func_list,
                    dummy_col='instance_id',time_col='time',
                    saving_path=None):
    if saving_path is not None:
        try:
            result = pd.read_pickle(saving_path)
        except:
            result = pd.DataFrame()
    counter = 0
    for user_index in range(len(user_list)):
        for ads_index in range(len(ads_list)):
            for chan_index in range(len(channel_list)):
                for hour_index in range(len(time_list)):
                    counter += 1
                    all_cols = user_list[user_index] + ads_list[ads_index] + channel_list[chan_index] + time_list[hour_index]
                
                    print('#######   {}  ########'.format(counter))
                    need_save_result = False
                    for func in func_list:
                        feature_name = get_new_name((user_index,ads_index,chan_index,hour_index),func)
                        
                        if feature_name in result.columns:
                            continue
                        if func.__name__ in [count_in_previous_n_time_unit.__name__,count_in_next_n_time_unit.__name__]:
                            col_time = all_cols.copy()
                            col_time.append(time_col)
                            if feature_name+'_10min' not in result.columns:
                                result[feature_name+'_10min'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*10})[feature_name]
                                need_save_result = True
                            if feature_name+'_60min' not in result.columns:
                                result[feature_name+'_60min'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*60})[feature_name]
                                need_save_result = True
                        elif func.__name__ in [time_to_n_next.__name__,time_to_n_previous.__name__]:
                            col_time = all_cols.copy()
                            col_time.append(time_col)
                            result[feature_name] = func(X,col_time,dummy_col,feature_name)[time_col]
                            need_save_result = True
                        else:
                            result[feature_name] = func(X,all_cols,dummy_col,feature_name)[feature_name]
                            need_save_result = True
                            
                    if need_save_result:
                        result['instance_id'] = X['instance_id'].copy()
                        result.to_pickle(saving_path)
    return result
                    
                    

In [ ]:
saving_path_count = FILE.X_fe_agg_count_formater.value.format('0915V1')
X = X.sort_values('time').reset_index(drop=True)
result = auto_generate_func(X,user_list,ads_list,channel_list,time_list,
                    count_func_list,
                    dummy_col='instance_id',time_col='time',
                    saving_path=saving_path_count)



#######   1  ########
#######   2  ########
#######   3  ########
#######   4  ########
#######   5  ########
#######   6  ########
#######   7  ########
#######   8  ########
#######   9  ########
#######   10  ########
#######   11  ########
#######   12  ########
#######   13  ########
#######   14  ########
#######   15  ########
#######   16  ########
#######   17  ########
#######   18  ########
#######   19  ########
#######   20  ########
#######   21  ########
#######   22  ########
#######   23  ########
#######   24  ########
#######   25  ########
#######   26  ########
#######   27  ########
#######   28  ########
#######   29  ########
#######   30  ########
#######   31  ########
#######   32  ########
#######   33  ########
#######   34  ########
#######   35  ########
#######   36  ########
#######   37  ########
#######   38  ########
#######   39  ########
#######   40  ########
#######   41  ########
#######   42  ########
#######   43  ########
#######   44  ######

In [14]:
saving_path_timeNext = FILE.X_fe_agg_time_next_formater.value.format('0915V1')
X = X.sort_values('time').reset_index(drop=True)
result_timeNext =auto_generate_func(X,user_list,ads_list,channel_list,time_list,
                    time_next_func_list,
                    dummy_col='instance_id',time_col='time',
                    saving_path=saving_path_timeNext)



#######   1  ########
#######   2  ########
#######   3  ########
#######   4  ########
#######   5  ########
#######   6  ########
#######   7  ########
#######   8  ########
#######   9  ########
#######   10  ########
#######   11  ########
#######   12  ########
#######   13  ########
#######   14  ########
#######   15  ########
#######   16  ########
#######   17  ########
#######   18  ########
#######   19  ########
#######   20  ########
#######   21  ########
#######   22  ########
#######   23  ########
#######   24  ########
#######   25  ########
#######   26  ########
#######   27  ########
#######   28  ########
#######   29  ########
#######   30  ########
#######   31  ########
#######   32  ########
#######   33  ########
#######   34  ########
#######   35  ########
#######   36  ########
#######   37  ########
#######   38  ########
#######   39  ########
#######   40  ########
#######   41  ########
#######   42  ########
#######   43  ########
#######   44  ######

In [16]:
result_timeNext['user4_ads4_channel3_hour1_time_to_n_previous'].describe()

count     10691.000000
mean    -123749.286877
std      123193.953709
min     -606818.000000
25%     -173744.500000
50%      -86698.000000
75%       -1767.500000
max           0.000000
Name: user4_ads4_channel3_hour1_time_to_n_previous, dtype: float64

In [17]:
saving_path_timeCount = FILE.X_fe_agg_time_count_formater.value.format('0915V1')
X = X.sort_values('time').reset_index(drop=True)
auto_generate_func(X,user_list,ads_list,channel_list,time_list,
                    count_time_func_list,
                    dummy_col='instance_id',time_col='time',
                    saving_path=saving_path_timeCount)

#######   1  ########


KeyboardInterrupt: 